In [1]:
#%reload 2
import matplotlib.pyplot as plt
import quandl
import pandas as pd
import os
import sys

In [2]:
quandl.ApiConfig.api_key = "_epxy8PNMMrBFS5JL4_B"

In [3]:
GAS_STATIONS_PATH = os.path.join('..', 'data', 'raw', 'input_data', 'Eingabedaten', 'Tankstellen.csv')
GAS_PRICE_PATH = os.path.join('..', 'data', 'raw', 'input_data', 'Eingabedaten', 'Benzinpreise')

In [4]:
start_date="2013-1-1"
end_date="2017-12-17"

In [5]:
gas_stations_df = pd.read_csv(GAS_STATIONS_PATH, sep=';', names=['id', 'Name', 'Company', 'Street', 'House_Number', 'Postalcode', 'City', 'Lat', 'Long'],index_col='id')

In [6]:
gas_station_id = 20

In [7]:
exchange = 'ODA/POILBRE_USD'
dataset = quandl.Dataset(exchange)
mydata = quandl.get(exchange, start_date=start_date, end_date=end_date)
exchange_data = mydata.resample('1T').mean().bfill()

In [ ]:
max_correlation_index = 0
max_correlation = float('-inf')
for gas_station_row in gas_stations_df.iterrows():
    try:
        gas_station_data = pd.read_csv(os.path.join(GAS_PRICE_PATH,'{}.csv'.format(gas_station_row[0])), names=['Timestamp', 'Price'],  index_col='Timestamp', parse_dates=['Timestamp'],sep=';')
        gas_station_data.resample('1T').mean().bfill()
        combined = exchange_data.merge(gas_station_data, left_index=True, right_index=True)
        correlation = combined.corr()['Price'][0]
        if max_correlation < correlation:
            max_correlation = correlation
            max_correlation_index = gas_station_row[0]
    except FileNotFoundError:
        pass

In [ ]:
max_correlation

In [ ]:
gas_stations_df.loc[max_correlation_index]

In [ ]:
gas_station_data = pd.read_csv(os.path.join(GAS_PRICE_PATH,'{}.csv'.format(max_correlation_index)), names=['Timestamp', 'Price'],  index_col='Timestamp', parse_dates=['Timestamp'],sep=';')

In [ ]:
fig, ax1 = plt.subplots()

ax1.plot(gas_station_data, 'r', alpha=0.5)
ax1.set_ylabel('Gasoline price in millicent at station ' + str(gas_station_id))

ax2 = ax1.twinx()
ax2.plot(mydata, 'b')
ax2.set_ylabel(dataset.name)

fig.autofmt_xdate()
plt.show()